In [ ]:
# You may also need to install torchvision and matplotlib
# !pip install matplotlib
# !pip install torchvision==0.11.0

# These might be useful if there are errors regarding ipywidgets while downloading torchvision.datasets
# !pip install ipywidgets
# !jupyter nbextension enable --py widgetsnbextension

## Imports

Importing `padl` and most importantly `transform` decorator used to change any `callable` to `padl.Transform`.

In [ ]:
import sys
sys.path.append('../')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torchvision
from torchvision import models

import padl
from padl import transform

## Dataset:
MNIST dataset available through torchvision is used in this notebook. The dataset can be separately downloaded from MNIST website or can be loaded as given below. 

More details on torchvision's MNIST dataset can be found here: https://pytorch.org/vision/stable/datasets.html#mnist

In [ ]:
mnist_train_dataset = torchvision.datasets.MNIST('data', train=True, download=True)
mnist_test_dataset = torchvision.datasets.MNIST('data', train=False, download=True)

<hr style="border:1px solid"> </hr>


### 1. Plot few images to check the data

`plot_image` is a normal standard function that takes in an image tensor and uses `matplotlib.pyplot` to plot the image. With `@transform` decorator, we can easily convert it to `padl.transform`. This allows us to use `padl` functional api and build data pipeline easily and quickly. 

Quick recap to `padl` operators:
- `>>`: Compose operator: $(f_1 >> f_2)(x) \rightarrow f_2(f_1(x))$
- `+`: Rollout operator: $(f_1 + f_2) (x) \rightarrow (f_1(x), f_2(x))$
- `/`: Parallel operator: $(f_1 / f_2)((x_1,x_2)) \rightarrow (f_1(x_1), f_2(x_2))$
- `-`: Name operator: Names a transform so that its output can be accesed by given name or the transform itself can be accessed by its name from the pipeline:  
    - $((f_1 - \text{'zulu'})+f_2)(x) \rightarrow \text{Namedtuple}(\text{'zulu'}:f_1(x), \text{'out_1'}:f_2(x))$
    - $((f_1 - \text{'zulu'})+f_2)[\text{'zulu'}] = f_1$
    
Note that outputs of `Rollout` and `Parallel` are tuples.

For more details on operators and building compound transforms, refer to the documentation: https://lf1-io.github.io/padl/gettingstarted.html#defining-compound-transforms

In [ ]:
@transform
def plot_image(img_tensor):
    fig= plt.figure(figsize=(2,2))
    ax = fig.add_subplot(111)
    ax.imshow(img_tensor, cmap='gray')
    plt.axis('off')
    return fig

### 1.1 Building a simple ploting pipeline using `padl` operators

Description of inbuilt transforms used.

- `padl.this` is a self reflexive trasform that allows for a quick mutation of input. 

        Example: padl.this[0]([1,2,3]) = 1

- `padl.Identity()` is a simple transform that does exactly as it sounds, passes the input on as it is. 

        Example: padl.Identity()([1,2,3]) = [1,2,3]



Description of `transform` pipeline defined below.
- `convert_plot`: Takes in a `PIL.Image` that is converted to numpy array, and then is plotted using `plot_image` transform defined above. 
- `plot_datapoint`: 
    - It is a `parrallel` that passes first part of input to convert_plot and passes second part of input as it is with `padl.Identity`
    - transforms are also named by `-`, so the output is a named tuple, with elements named as the transform name.




In [ ]:
@transform
def img_to_array(img):
    return np.asarray(img)

convert_plot = (
    img_to_array
    >> plot_image
)

plot_datapoint = (convert_plot - 'image')/ (padl.Identity() - 'label')

In [ ]:
out = plot_datapoint(mnist_train_dataset[0])

In [ ]:
out.label

In [ ]:
# Plotting some of the train datapoints

for _ in range(5):
    output = plot_datapoint(mnist_train_dataset[np.random.randint(len(mnist_train_dataset))])
    print(f'Label : {output.label}')
    # showing image
    plt.show()
    print('-'*30)

<hr style="border:1px solid"> </hr>


### 2. Model
We will build a simple `Unet` to classify `MNIST` handwritings. In the cell below, a simple pytorch net is defined with just one added decorator `@transform`. This is enough to wrap the pytorch model into `padl.Transform` and use it with other transform to build a data pipeline.

### 2.1 Simple Unet

In [ ]:

import torch.nn.functional as F
from torch.utils.data import DataLoader
import torch.optim as optim
from torch.optim import lr_scheduler

        
@transform
class SimpleNet(torch.nn.Module):
    def __init__(self):
        super().__init__()

        # Conv 1
        # size : input: 28x28x1 -> output : 26 x 26 x 32
        self.conv1 = torch.nn.Conv2d(1, 32, kernel_size=3)
        self.batchnorm1 = torch.nn.BatchNorm2d(32)
        
        # Conv 2
        # size : input: 26x26x32 -> output : 24 x 24 x 32
        self.conv2 = torch.nn.Conv2d(32, 32, kernel_size=3)
        self.batchnorm2 = torch.nn.BatchNorm2d(32)
        
        # Conv 3
        # size : input: 24x24x32 -> output : 12 x 12 x 32
        self.conv3 = torch.nn.Conv2d(32, 32, kernel_size=2, stride = 2)
        self.batchnorm3 = torch.nn.BatchNorm2d(32)
        
        # Conv 4
        # size : input : 12 x 12 x 32 -> output : 8 x 8 x 64
        self.conv4 = torch.nn.Conv2d(32, 64, kernel_size=5)
        self.batchnorm4 = torch.nn.BatchNorm2d(64)
        
        # Conv 5
        # size : input: 8x8x64 -> output : 4 x 4 x 64 -> Linearize = 1024
        self.conv5 = torch.nn.Conv2d(64, 64, kernel_size=2, stride = 2)
        self.batchnorm5 = torch.nn.BatchNorm2d(64)
        
        # dropout layer 
        self.conv5_drop = torch.nn.Dropout2d()
        
        # FC 1 
        self.fc1 = torch.nn.Linear(1024, 128)
        
        # FC 2
        self.fc2 = torch.nn.Linear(128, 10)

    def forward(self, x):
        x = self.batchnorm1(F.relu(self.conv1(x)))
        x = self.batchnorm2(F.relu(self.conv2(x)))
        x = self.batchnorm3(F.relu(self.conv3(x)))
        x = self.batchnorm4(F.relu(self.conv4(x)))
        x = self.batchnorm5(F.relu(self.conv5(x)))
        x = self.conv5_drop(x)
        x = x.view(-1, 1024)
        x = F.relu(self.fc1(x))
        x = F.log_softmax(self.fc2(x), dim=1)
        return x

### 2.2 Preprocessing

The `preprocess` pipeline below, again splits the datapoint to two different pipelines for `PIL.Image` and `int` label. First part of parralel pipeline, converts the image into torch tensor of type float, and second part passes the label as it is. The label later will be used for the loss function. 

In [ ]:
@transform
def convert_to_tensor(img):
    arr = np.asarray(img)
    return torch.tensor(arr).type(torch.FloatTensor)


preprocess = (
    convert_to_tensor / convert_to_tensor
    >> padl.this.reshape(-1, 28, 28) / padl.Identity()
)

### 2.3 Instantiating the network and loss function

Initialising instances of `SimpleNet` and `loss` function. Loss function here is a wrapped `torch` negative log likelihood loss which is again wrapped easily with same `transform` call. 

In [ ]:
simplenet = SimpleNet()
loss_func = transform(F.nll_loss)

In [ ]:
simplenet

In [ ]:
loss_func

### 2.4  Building the training model

`train_model` is now composed (`>>`) with the transforms already defined.
- preprocess: preprocessing transform defined above
- Batchify: Batchify is a inbuilt `transform` that marks end of preprocess (dataloading) and that adds batch dimension to the inputs. Batchify also moves the input tensors to device specified for the model
- simplenet: Instance of SimpleNet
- padl.this: A self reflexive trasform that allows for a quick mutation of input.

`train_model` is then sent to the intended device. It is by default in `cpu`.


In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Device to be used: ', device)

In [ ]:
train_model = (
    preprocess
    >> padl.Batchify()
    >> simplenet / padl.this.type(torch.long)
)

train_model.pd_to(device)

<hr style="border:1px solid"> </hr>


### 3. Training and validating the `train_model`

Training is not much different than the normal torch training steps, except dataloading and training is made even simplier by `train_apply`. It is one of the three inbuilt methods along with `infer_apply` and `eval_apply` that handles the stage context of model.

In [ ]:
learning_rate = 0.01
momentum = 0.5
log_interval = 10
nepoch = 2
num_workers = 0

optimizer = optim.SGD(train_model.pd_parameters(), lr=learning_rate, momentum = momentum)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.95)



for epoch in range(nepoch):
    step_counter = 0
    for batch_output, batch_targets in train_model.train_apply(mnist_train_dataset, num_workers=num_workers, batch_size=256):

        optimizer.zero_grad()

        loss = F.nll_loss(batch_output, batch_targets)

        loss.backward()

        optimizer.step()
        exp_lr_scheduler.step()

        if step_counter % log_interval == 0:
            print(f'Epoch:{epoch}; Step: {step_counter}; loss: {loss}')
        step_counter += 1

### 3.1 Accuracy of the model

We can quickly build a `validation_model` by adding a further step to `train_model` to get the number associated with the maximum confidence predicted by the model. 


Note: As we don't have a separate validation dataset with labels, we will have to use the same train data to `validate` the model in this example.


First, lets look at the format of the prediction by `infer_apply`ing one of datapoint.

In [ ]:
train_model

In [ ]:
train_model.infer_apply(mnist_train_dataset[0])

`train_model` predicts a tensor of confidence associated for the 10 numbers, and the index associated with the maximum of these confidence is the prediction by the model. Thus, we can add another transform to the same `train_model` to get that index associated with maximum of the confidence. 

Note that the new `validation_model` is a new instance of Transform but it contains same objects as `train_model` with added two new `transform`s. All the transform objects that are already in `train_model` is in `device` as assigned above, but the main `validation_model` object itself will have by default `cpu` device assigned. Thus, to move it to (or assign it with) correct device, we have to again call `pd_to(device)`.

In [ ]:
validation_model = (
    train_model
    >> padl.transform(lambda x: x.max(1).indices) / padl.Identity()
)

# We need to send the validation_model to device again
validation_model.pd_to(device)

In [ ]:
# Compute accuracy over the test dataset

accuracy = 0
for batch_output, batch_targets in validation_model.eval_apply(mnist_test_dataset, num_workers=0, batch_size=256):
    accuracy += (batch_targets == batch_output).sum()

accuracy = accuracy.item()/ len(mnist_test_dataset)
print(f'Accuracy of model: {accuracy}')

Not a bad accuracy of `~0.95` for a quick train model.

### 3.2 Infer few images from test data

Although we do not have labels for images in test data, we can still infer and verify the predictions ourselves. For that, we can again use model object `simplenet` that we have trained by using `train_model` and now stack it with other `transform`s to build an infer model.

In [ ]:
infer_preprocess =(
    padl.this[0]
    >> convert_to_tensor
)
infer_model = (
    infer_preprocess
    >> padl.Batchify()
    >> padl.this.unsqueeze(1) 
    >> simplenet
    >> padl.transform(lambda x: x.max(1).indices)
)

infer_model.pd_to(device)

In [ ]:
for _ in range(5):
    data_point = mnist_test_dataset[np.random.randint(len(mnist_test_dataset))]
    output = plot_datapoint(data_point)
    plt.show()
    print(f'Prediction: {infer_model.infer_apply(data_point).item()}')
    print('-'*30)

<hr style="border:1px solid"> </hr>


### 4. Using further image augmentation on training

We can easily use some of the `torchvision.transforms` for image augmentation and add it to our preproccessing of image to help with training. Lets add a couple of augmentations to our training: `GaussianBlur` and `RandomRotation`
We need to wrap the call to these `torchvision.transforms` with our `padl.transform` before instantiating them, and that is all. 

In [ ]:
import torchvision.transforms as T

In [ ]:
gaussian_blur = transform(T.GaussianBlur)(kernel_size=(3,3), sigma=0.1)

rotate_img = transform(T.RandomRotation)(degrees=(-15,15))

Now we can again use `padl`'s functional api to build an `image_augmentation` pipeline.

Note: `torchvision.transforms` expect images with channels but our images are just in grayscale. So, we need to unsqueeze our image tensor here.

In [ ]:
image_augmentation = (
    padl.this.unsqueeze(0)
    >> rotate_img
    >> gaussian_blur
    >> padl.this[0]
)

####  Sample of image augmentation

Lets try the augmentation on one of image. 

In [ ]:
datapoint = mnist_train_dataset[2]

# plot datapoint 
plot_datapoint(datapoint)

# convert datapoint's image to tensor
img_tensor = convert_to_tensor(datapoint[0])

# tensor shape
print('tensor shape:', img_tensor.shape)

Augmented images: Running image augmentation on same image tensor

In [ ]:
for _ in range(5):
    out_aug = image_augmentation(img_tensor)
    plot_image(out_aug)

### 4.2 We can add the `image_augmentation` pipeline easily to the `preprocess` and rebuild `train_model`

In [ ]:
preprocess_with_augmentation = (
    convert_to_tensor / convert_to_tensor
    >> padl.this.reshape(-1, 28, 28) / padl.Identity()
    >> image_augmentation / padl.Identity()
)

train_model = (
    preprocess_with_augmentation
    >> padl.Batchify()
    >> simplenet / padl.this.type(torch.long)
)


train_model.pd_to(device)

### 4.3 Retraining the model with `image_augmentation`

In [ ]:
learning_rate = 0.01
momentum = 0.5
log_interval = 10
nepoch = 3
num_workers = 0

optimizer = optim.SGD(train_model.pd_parameters(), lr=learning_rate, momentum = momentum)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.95)



for epoch in range(nepoch):
    step_counter = 0
    for batch_output, batch_targets in train_model.train_apply(mnist_train_dataset, num_workers=num_workers, batch_size=256):

        optimizer.zero_grad()

        loss = F.nll_loss(batch_output, batch_targets)

        loss.backward()

        optimizer.step()
        exp_lr_scheduler.step()

        if step_counter % log_interval == 0:
            print(f'Epoch:{epoch}; Step: {step_counter}; loss: {loss}')
        step_counter += 1

### 4.4 Validate the model

In [ ]:
# Calculate accuracy
accuracy = 0
for batch_output, batch_targets in validation_model.eval_apply(mnist_test_dataset, num_workers=0, batch_size=256):
    accuracy += (batch_targets == batch_output).sum()

accuracy = accuracy.item()/ len(mnist_test_dataset)
print(f'Accuracy of model: {accuracy}')